In [36]:
import pandas as pd
import os.path
from nltk.tokenize import word_tokenize
import nltk

DATAPATH = "data/trial"

class exerpt:
    def __init__(self, name, txt, ann, tsv, grobid):
        self.name = name
        self.txt = txt
        self.ann = ann
        self.tsv = tsv
        self.grobid = grobid
        self.context = False
        self.posTag = False
        
    def getContext(self, count = 3):
        if self.context == False: 
            beforeContext = []
            afterContext = []
            self.context = True
            for start,end in zip(self.tsv["startOffset"].values, self.tsv["endOffset"].values ):#self.tsv.iterrows():
                a = getIndexSeperated(exerpt.txt[0],end,count = count)
                b = getIndexSeperated(exerpt.txt[0],start,count = count,forward = False)
                beforeContext.append(self.txt[0][b:start])
                afterContext.append(self.txt[0][end:a])
            self.tsv.insert(3,"beforeContext", beforeContext)
            self.tsv.insert(3,"afterContext", afterContext)
            
    def getPosTag(self):
        if self.context and self.posTag == False: 
            bTag = []
            aTag = []
            self.posTag = True
            for before, after in zip(self.tsv["beforeContext"].values,self.tsv["afterContext"].values):
                bTag.append(nltk.pos_tag(word_tokenize(before)))
                aTag.append(nltk.pos_tag(word_tokenize(after)))
            self.tsv.insert(3,"beforeTag", bTag)
            self.tsv.insert(3,"afterTag", aTag)
            


def readTXTByLine(filepath):
    ftemp = open(filepath, "r", encoding = "utf-8")
    raw = []
    for line in ftemp: 
        raw.append(line)
    ftemp.close()
    return raw

data = {}

with open("data/trial/prefixes.txt","r") as f:
    for l in f:
        l = l[:-1]
        data[l] = exerpt(
            l,
            readTXTByLine(os.path.join(DATAPATH, "txt", l + ".txt")),
            readTXTByLine(os.path.join(DATAPATH, "ann", l + ".ann")),
            pd.read_csv(os.path.join(DATAPATH, "tsv", l + ".tsv"), "\t", header = 0 ),
            json.load(open(os.path.join(DATAPATH, "grobid", l + ".grobid")))
        )

In [37]:
list(data.values())

In [38]:
for exerpt in list(data.values()):
    print("Name:", exerpt.name)
    print("grobid")
    for x in exerpt.grobid[1]["measurements"]:
        if x["type"] == "value":
            print(x["quantity"]["offsetStart"],"-",x["quantity"]["offsetEnd"],":",x["quantity"]["rawValue"])
        elif x["type"] == "listc":
            for q in x["quantities"]:
                print(q["offsetStart"],"-",q["offsetEnd"],":",q["rawValue"])
        elif x["type"] == "interval":
            try:
                print(x["quantityLeast"]["offsetStart"],"-",x["quantityLeast"]["offsetEnd"],":",x["quantityLeast"]["rawValue"])
            except KeyError:
                continue
            try:
                print(x["quantityMost"]["offsetStart"],"-",x["quantityMost"]["offsetEnd"],":",x["quantityMost"]["rawValue"])
            except KeyError:
                continue
    print("gold")            
    for start, end, text, tpe in  zip(exerpt.tsv["startOffset"].values, exerpt.tsv["endOffset"].values,exerpt.tsv["text"].values,exerpt.tsv["annotType"].values):
        if tpe == "Quantity":
            print(start,"-", end, ":", text)
        

Name: S0012821X12004384-1302
grobid
492 - 497 : three
553 - 557 : Five
588 - 595 : 2619.60
597 - 604 : 2617.35
606 - 613 : 2617.44
615 - 622 : 2614.73
628 - 635 : 2614.71
769 - 776 : 2619.60
778 - 785 : 2614.73
791 - 798 : 2614.71
1481 - 1484 : two
1493 - 1500 : 2619.60
1505 - 1512 : 2614.71
1992 - 1995 : two
2115 - 2118 : two
gold
553 - 557 : Five
588 - 637 : 2619.60, 2617.35, 2617.44, 2614.73, and 2614.71 m
769 - 800 : 2619.60, 2614.73, and 2614.71 m
1481 - 1484 : two
1493 - 1512 : 2619.60 and 2614.71
Name: S0012821X12004384-1405
grobid
626 - 628 : 20
775 - 776 : 0
781 - 783 : 80
gold
626 - 628 : 20
775 - 784 : 0% to 80%
Name: S0012821X12004384-1415
grobid
44 - 48 : 2632
52 - 56 : 2618
291 - 292 : 5
gold
39 - 58 : from 2632 to 2618 m
280 - 293 : on average 5%
Name: S0012821X12004384-1594
grobid
162 - 166 : 2618
310 - 312 : 40
gold
156 - 168 : above 2618 m
310 - 314 : 40°N
Name: S0012821X12004384-1599
grobid
81 - 84 : 103
117 - 121 : 2618
gold
76 - 88 : from 103 yrs
111 - 123 : above 

884 - 887 : 490
988 - 991 : 430
1014 - 1016 : 60
gold
842 - 854 : 74°S to 90°S
884 - 889 : 490 K
988 - 993 : 430 K
1014 - 1018 : 60 K
Name: S0019103512003995-1807
grobid
323 - 328 : 589.3
421 - 425 : four
753 - 760 : 300-800
846 - 847 : 3
gold
323 - 331 : 589.3 nm
421 - 425 : four
753 - 763 : 300–800 nm
846 - 852 : 3 mbar
Name: S0019103512003995-1910
grobid
112 - 116 : four
123 - 127 : half
149 - 153 : 1947
158 - 162 : 1999
211 - 212 : 1
218 - 220 : 37
229 - 232 : one
259 - 261 : 27
304 - 307 : one
339 - 340 : 3
343 - 344 : 15
557 - 562 : three
618 - 622 : half
663 - 664 : 2
gold
112 - 127 : four and a half
149 - 162 : 1947 and 1999
211 - 221 : 1% and 37%
229 - 232 : one
259 - 266 : 27 days
304 - 307 : one
339 - 345 : 9 ± 6%
493 - 504 : ∼10–11 days
557 - 562 : three
597 - 622 : within a month and a half
649 - 666 : approximately 2 h
Name: S0019103512003995-2760
grobid
324 - 326 : 11
337 - 338 : 5
gold
324 - 333 : 11 km s−1
337 - 340 : 5Rp
Name: S0019103512003995-3548
grobid
235 - 240 :

In [2]:
        
def getIndexSeperated(text, index, count=3, char=" ", forward=True):
    """
    Description: method that searches for the next index(depending on direction), where 
    there is an occurence of exactly count words between the current word
    
    count: number of characters to look for 
    text: the string to look through
    index: the index of the string to start at
    """
    if forward == True: 
        r = range(index, len(text))
    else:
        r = reversed(range(0, index))
    
    charCount = 0
    for i in r:
        if text[i] == " ":
            charCount += 1
        
        if charCount == count + 1:
            return i
        
    if forward == True:
        return len(text) - 1
    else :
        return 0

    

In [3]:
count =1
h = []
for exerpt in data.values():
    exerpt.getContext(7)
    exerpt.getPosTag()
#     for l in exerpt.tsv.iterrows():
#         if count ==1 :
#             print(l[1][4])
#             a = getIndexSeperated(exerpt.txt[0],l[1][4],count = 7)
#             b = getIndexSeperated(exerpt.txt[0],l[1][3],count = 7,forward = False)
#             h.append(exerpt.txt[0][b:a])
    count = 90

In [4]:
for column in data["S0012821X12004384-1302"].tsv["beforeTag"].values:
    print(column[0])

('axes', 'NNS')
('(', '(')
('Five', 'CD')
('(', '(')
('highest', 'JJS')
('3', 'CD')
('(', '(')
('near', 'IN')
('the', 'DT')
('younger', 'JJR')
('the', 'DT')


In [5]:
qCounts = {}
mCounts = {}
qText = []
mText = []
for exerpt in data.values():
    for text, annotType, bTag, aTag in zip(exerpt.tsv["text"].values, exerpt.tsv["annotType"].values,exerpt.tsv["beforeTag"].values,exerpt.tsv["afterTag"].values):
        if(annotType == "Quantity"):
            bCount = 0
            num = 1
            while(bCount < len(bTag)):
                #if bTag[len(bTag) - bCount - 1][0].isalnum():
                try:
                    qCounts["b" + str(num)].append(bTag[len(bTag) - bCount - 1][1])
                    num += 1
                except KeyError:
                    qCounts["b" + str(num)] = [bTag[len(bTag) - bCount - 1][1]]
                    num += 1
                bCount +=1
                
            aCount = 0
            num = 1
            while(aCount < len(aTag)):
                #if aTag[aCount][0].isalnum():
                try:
                    qCounts["a" + str(num)].append(aTag[aCount][1])
                    num += 1
                except KeyError:
                    qCounts["a" + str(num)] = [aTag[aCount][1]]
                    num += 1
                aCount +=1
                
            qText.append(text)
                    
        elif(annotType == "MeasuredEntity"):
            bCount = 0
            num = 1
            while(bCount < len(bTag)):
                #if bTag[len(bTag) - bCount - 1][0].isalnum():
                try:
                    mCounts["b" + str(num)].append(bTag[len(bTag) - bCount - 1][1])
                    num += 1
                except KeyError:
                    mCounts["b" + str(num)] = [bTag[len(bTag) - bCount - 1][1]]
                    num += 1
                bCount +=1
                
            aCount = 0
            num = 1
            while(aCount < len(aTag)):
                #if aTag[aCount][0].isalnum():
                try:
                    mCounts["a" + str(num)].append(aTag[aCount][1])
                    num += 1
                except KeyError:
                    mCounts["a" + str(num)] = [aTag[aCount][1]]
                    num += 1
                aCount +=1
            
            mText.append(text)
        
            
    

In [6]:
qCounts

{'b1': ['.',
  'IN',
  '(',
  'DT',
  'NNS',
  'IN',
  'IN',
  ',',
  'VBZ',
  'NN',
  'VBZ',
  'JJ',
  '(',
  'NN',
  'NN',
  'RB',
  'DT',
  'VBD',
  'DT',
  'CC',
  'IN',
  'IN',
  'IN',
  'IN',
  'TO',
  'TO',
  'IN',
  'IN',
  'CC',
  'VBD',
  'IN',
  'NN',
  '(',
  'IN',
  '(',
  'IN',
  'TO',
  'DT',
  'NNS',
  '(',
  'IN',
  'DT',
  'CC',
  'NN',
  'DT',
  '(',
  '(',
  'IN',
  'IN',
  'IN',
  'DT',
  '(',
  'DT',
  'DT',
  'NN',
  'IN',
  ',',
  'TO',
  'NNP',
  'NN',
  'DT',
  '(',
  'VBD',
  'IN',
  'IN',
  '(',
  'IN',
  'IN',
  'VB',
  '(',
  '(',
  'VBN',
  'VBG',
  'IN',
  'NNS',
  'DT',
  '(',
  'IN',
  'DT',
  'IN',
  'NNP',
  'NNP',
  'CC',
  'JJ',
  'IN',
  'IN',
  'NNS',
  'IN',
  'IN',
  'IN',
  'IN',
  'IN',
  'CC',
  'IN',
  'IN',
  'IN',
  'IN',
  'IN',
  'TO',
  'VBD',
  'NNS',
  'IN',
  'TO',
  'TO',
  'NNS',
  ',',
  'CC',
  'IN',
  'IN',
  'IN',
  'IN',
  'IN',
  'IN',
  'IN',
  '(',
  ',',
  'TO',
  'NNS',
  'IN',
  'TO',
  'TO',
  'IN',
  ',',
  'CC',
  'I

In [7]:
for k in qCounts.keys():
    qCounts[k] = nltk.FreqDist(qCounts[k])
    
for k in mCounts.keys():
    mCounts[k] = nltk.FreqDist(mCounts[k])

In [8]:
qCounts

{'b1': FreqDist({'IN': 120, 'DT': 24, '(': 23, 'NN': 23, 'CC': 18, 'TO': 14, 'NNS': 10, ',': 10, 'VBZ': 10, 'NNP': 9, ...}),
 'b2': FreqDist({'NN': 98, 'NNS': 35, 'IN': 28, 'NNP': 24, 'CD': 23, 'VBN': 16, ')': 6, 'RB': 6, 'DT': 6, 'TO': 6, ...}),
 'b3': FreqDist({'NN': 43, 'JJ': 40, 'DT': 33, 'IN': 31, 'NNP': 30, 'CD': 17, 'NNS': 17, 'VBN': 12, 'VBD': 9, '(': 7, ...}),
 'b4': FreqDist({'IN': 60, 'NN': 47, 'DT': 46, 'JJ': 25, 'CD': 21, 'NNP': 16, 'NNS': 12, 'VBD': 12, 'VBZ': 8, '.': 7, ...}),
 'b5': FreqDist({'NN': 54, 'IN': 49, 'DT': 30, 'NNS': 24, 'NNP': 19, 'JJ': 16, ',': 11, 'RB': 10, 'CC': 9, '.': 8, ...}),
 'b6': FreqDist({'NN': 61, 'IN': 37, 'DT': 25, 'NNP': 22, 'JJ': 20, 'CD': 20, 'NNS': 18, ')': 12, '.': 10, ',': 9, ...}),
 'b7': FreqDist({'NN': 51, 'IN': 37, 'DT': 36, 'CD': 28, 'NNP': 24, 'JJ': 20, ',': 13, 'NNS': 12, 'VBN': 9, 'VBZ': 9, ...}),
 'b8': FreqDist({'NN': 25, 'CD': 17, 'IN': 17, 'NNP': 11, 'NNS': 10, ',': 9, 'RB': 9, 'DT': 9, 'JJ': 8, 'VBN': 7, ...}),
 'b9': FreqDi

In [9]:
qText = [nltk.pos_tag(word_tokenize(text)) for text in qText]
mText = [nltk.pos_tag(word_tokenize(text)) for text in mText]

In [10]:
mText

[[('samples', 'NNS')],
 [('samples', 'NNS')],
 [('samples', 'NNS')],
 [('samples', 'NNS')],
 [('samples', 'NNS')],
 [('specimens', 'NNS')],
 [('Samples', 'NNS')],
 [('Dinoflagellate', 'NNP'),
  ('cyst', 'NN'),
  ('assemblage', 'NN'),
  ('1', 'CD'),
  ('(', '('),
  ('DA1', 'NNP')],
 [('DA1', 'NN')],
 [('CIE', 'NNP'), ('onset', 'NN')],
 [('northern', 'JJ'), ('rain', 'NN'), ('belt', 'NN')],
 [('CIE', 'NNP'), ('onset', 'NN')],
 [('CIE', 'NNP'), ('onset', 'NN')],
 [('Oi1', 'NN')],
 [('ice', 'NN'), ('volume', 'NN')],
 [('ice', 'NN'), ('volume', 'NN')],
 [('Oi1', 'NN')],
 [('timescale', 'NN')],
 [('decline', 'NN')],
 [('fraction', 'NN')],
 [('spicules', 'NNS')],
 [('concentrated', 'VBN'), ('HNO3', 'NNP')],
 [('wet', 'JJ'), ('alkaline', 'NN'), ('digestion', 'NN')],
 [('wet', 'JJ'), ('alkaline', 'NN'), ('digestion', 'NN')],
 [('wet', 'JJ'), ('alkaline', 'NN'), ('digestion', 'NN')],
 [('wet', 'JJ'), ('alkaline', 'NN'), ('digestion', 'NN')],
 [('samples', 'NNS')],
 [('thermally', 'RB'), ('distill

In [11]:
txt = {"textQ":[pos[1] for n in qText for pos in n],"textM":[pos[1] for n in mText for pos in n]}

In [12]:
for k in txt.keys():
    txt[k] = nltk.FreqDist(txt[k])


In [13]:
txt

{'textQ': FreqDist({'CD': 302, 'NN': 204, 'NNP': 41, 'IN': 33, 'TO': 20, 'NNS': 20, 'RB': 20, 'CC': 18, 'JJ': 17, ',': 9, ...}),
 'textM': FreqDist({'NN': 250, 'NNS': 78, 'NNP': 76, 'JJ': 76, 'IN': 30, 'DT': 13, 'CD': 9, 'VBN': 6, 'RB': 6, 'VBZ': 5, ...})}

In [14]:
start, end = (data["S0012821X12004384-1302"].tsv["startOffset"].values,data["S0012821X12004384-1302"].tsv["endOffset"].values)

In [15]:
for start, end in zip(data["S0012821X12004384-1302"].tsv["startOffset"].values,data["S0012821X12004384-1302"].tsv["endOffset"].values):
    print(start)

553
558
588
558
571
769
760
1481
1485
1493
1485


In [16]:
data["S0012821X12004384-1302"].tsv

,docId,annotSet,annotType,afterTag,beforeTag,afterContext,beforeContext,startOffset,endOffset,annotId,text,other
0,S0012821X12004384-1302,1,Quantity,"[(samples, NNS), (from, IN), (below, IN), (the...","[(axes, NNS), ((, (), (describing, VBG), (the,...","samples from below the CIE at 2619.60,",axes (describing the highest percentages of v...,553,557,T9,Five,"{mods: [IsCount], unit: count}"
1,S0012821X12004384-1302,1,MeasuredEntity,"[(from, IN), (below, IN), (the, DT), (CIE, NNP...","[((, (), (describing, VBG), (the, DT), (highes...","from below the CIE at 2619.60, 2617.35,",(describing the highest percentages of varian...,558,565,T10,samples,{HasQuantity: T9}
2,S0012821X12004384-1302,2,Quantity,"[((, (), (indicated, VBN), (in, IN), (Fig, NNP...","[(Five, CD), (samples, NNS), (from, IN), (belo...","(indicated in Fig. 5) contain Apectodinium, in",Five samples from below the CIE at,588,637,T1,"2619.60, 2617.35, 2617.44, 2614.73, and 2614.71 m","{mods: [IsList], unit: m}"
3,S0012821X12004384-1302,2,MeasuredEntity,"[(from, IN), (below, IN), (the, DT), (CIE, NNP...","[((, (), (describing, VBG), (the, DT), (highes...","from below the CIE at 2619.60, 2617.35,",(describing the highest percentages of varian...,558,565,T10,samples,{HasQuantity: T1}
4,S0012821X12004384-1302,2,Qualifier,"[(at, IN), (2619.60, CD), (,, ,), (2617.35, CD...","[(highest, JJS), (percentages, NNS), (of, IN),...","at 2619.60, 2617.35, 2617.44, 2614.73, and 26...",highest percentages of variance). Five sample...,571,584,T15,below the CIE,{Qualifies: T1}
5,S0012821X12004384-1302,3,Quantity,"[(), )), (also, RB), (contain, VBP), (negative...","[(3, CD), (and, CC), (7, CD), (), )), (., .), ...",") also contain negative δ13CTOC, indicative of...",3 and 7). Some of these samples (,769,800,T2,"2619.60, 2614.73, and 2614.71 m","{mods: [IsList], unit: m}"
6,S0012821X12004384-1302,3,MeasuredEntity,"[((, (), (2619.60, CD), (,, ,), (2614.73, CD),...","[((, (), (Figs, NNP), (., .), (3, CD), (and, C...","(2619.60, 2614.73, and 2614.71 m) also contain",(Figs. 3 and 7). Some of these,760,767,T11,samples,{HasQuantity: T2}
7,S0012821X12004384-1302,4,Quantity,"[(samples, NNS), (2619.60, CD), (and, CC), (26...","[(near, IN), (the, DT), (younger, JJR), (sampl...",samples 2619.60 and 2614.71 have a spore,near the younger samples from PA4. The,1481,1484,T3,two,"{mods: [IsCount], unit: count}"
8,S0012821X12004384-1302,4,MeasuredEntity,"[(2619.60, CD), (and, CC), (2614.71, CD), (hav...","[(the, DT), (younger, JJR), (samples, NNS), (f...",2619.60 and 2614.71 have a spore and,the younger samples from PA4. The two,1485,1492,T14,samples,{HasQuantity: T3}
9,S0012821X12004384-1302,5,Quantity,"[(have, VB), (a, DT), (spore, NN), (and, CC), ...","[(younger, JJR), (samples, NNS), (from, IN), (...",have a spore and pollen palynological signature,younger samples from PA4. The two samples,1493,1512,T13,2619.60 and 2614.71,"{mods: [IsList], unit: Unspecified}"


In [17]:
data["S0012821X12004384-1302"].txt[0][553:557]

'Five'

In [18]:
data["S0012821X12004384-1302"].txt

['Correspondence analysis (CA) and statistical diversity analysis were carried out on the palynological dataset (total counts per gram) to confirm assemblage designations (Figs. 4 and 5), to identify any disturbance to the core prior to interpretation, and to estimate diversity (Fig. 6). Dinoflagellate cyst assemblages (DA1–DA5) and pollen assemblages (PA1–PA4) were defined by visually comparing changes in the species dominance (Figs. 7 and 8), and confirmed by CA (Fig. 4) using the first three axes (describing the highest percentages of variance). Five samples from below the CIE at 2619.60, 2617.35, 2617.44, 2614.73, and 2614.71 m (indicated in Fig. 5) contain Apectodinium, in contrast to the other samples below the CIE (Figs. 3 and 7). Some of these samples (2619.60, 2614.73, and 2614.71 m) also contain negative δ13CTOC, indicative of the CIE (Fig. 3). To test if coincident pollen and spore changes also occur in these samples, we used CA on the pollen and spore data only (Fig. 5a). P

In [19]:
data["S0012821X12004384-1302"].ann

['T1\tQuantity 588 637\t2619.60, 2617.35, 2617.44, 2614.73, and 2614.71 m\n',
 'T2\tQuantity 769 800\t2619.60, 2614.73, and 2614.71 m\n',
 'T9\tQuantity 553 557\tFive\n',
 'T10\tMeasuredEntity 558 565\tsamples\n',
 'T11\tMeasuredEntity 760 767\tsamples\n',
 'R7\tHasQuantity Arg1:T11 Arg2:T2\t\n',
 'T13\tQuantity 1493 1512\t2619.60 and 2614.71\n',
 'T14\tMeasuredEntity 1485 1492\tsamples\n',
 'R10\tHasQuantity Arg1:T14 Arg2:T13\t\n',
 "#5\tAnnotatorNotes T10\tProperty for the quantities is probably 'depth'\n",
 'A1\tIsCount T9\n',
 'R11\tHasQuantity Arg1:T10 Arg2:T9\t\n',
 'T15\tQualifier 571 584\tbelow the CIE\n',
 'A2\tQuantityQualifier T1 IsList\n',
 '#1\tAnnotatorNotes T1\tm\n',
 'R1\tQualifies Arg1:T15 Arg2:T1\t\n',
 'R2\tHasQuantity Arg1:T10 Arg2:T1\t\n',
 'A3\tQuantityQualifier T2 IsList\n',
 '#2\tAnnotatorNotes T2\tm\n',
 'T3\tQuantity 1481 1484\ttwo\n',
 'A4\tIsCount T3\n',
 'R3\tHasQuantity Arg1:T14 Arg2:T3\t\n',
 'A5\tQuantityQualifier T13 IsList\n',
 '#3\tAnnotatorNotes T13\

In [20]:
import nltk
from nltk.tokenize import word_tokenize
tokens = word_tokenize(data["S0012821X12004384-1302"].txt[0])
#tokens = [tok for tok in tokens if tok.isalnum()]

In [21]:
tagged = nltk.pos_tag(tokens)
len(tagged)

424

In [22]:
def process(words):
    try: 
        chunkgram = r"""measurement : {<NN>*<IN>+<CD|NN|CC>+<IN>+}
                                    {<NN>+<,|(|)>*<CD|NN|CC>+<,|(|)>*<IN>*}
                                    {<NN>+<IN>+<CD|NN|CC>+<IN>+}
                                    {<>+<CD|NN|CC>+<IN>+}
                            """
        chunkparser = nltk.RegexpParser(chunkgram)
        chunked = chunkparser.parse(tagged)
            
        print(chunked)
        print("==============")
        
        chunked.draw()
            
            
    except Exception as e: 
        print(str(e))
        
        
process(tagged)

(S
  (measurement Correspondence/NN analysis/NN)
  (/(
  CA/NNP
  )/)
  and/CC
  statistical/JJ
  (measurement diversity/NN analysis/NN)
  were/VBD
  carried/VBN
  out/RP
  on/IN
  the/DT
  palynological/JJ
  dataset/NN
  (/(
  total/JJ
  counts/NNS
  per/IN
  gram/NN
  )/)
  to/TO
  confirm/VB
  assemblage/NN
  designations/NNS
  (/(
  Figs/NNP
  ./.
  4/CD
  and/CC
  5/CD
  )/)
  ,/,
  to/TO
  identify/VB
  any/DT
  disturbance/NN
  to/TO
  the/DT
  core/NN
  prior/RB
  to/TO
  (measurement interpretation/NN ,/, and/CC)
  to/TO
  estimate/VB
  diversity/NN
  (/(
  Fig/NNP
  ./.
  6/CD
  )/)
  ./.
  Dinoflagellate/NNP
  cyst/NN
  assemblages/NNS
  (/(
  DA1–DA5/NNP
  )/)
  and/CC
  pollen/VBN
  assemblages/NNS
  (/(
  PA1–PA4/NNP
  )/)
  were/VBD
  defined/VBN
  by/IN
  visually/RB
  comparing/VBG
  changes/NNS
  in/IN
  the/DT
  species/NNS
  dominance/NN
  (/(
  Figs/NNP
  ./.
  7/CD
  and/CC
  8/CD
  )/)
  ,/,
  and/CC
  confirmed/VBN
  by/IN
  CA/NNP
  (/(
  Fig/NNP
  ./.
  4/CD
 

In [23]:
"10002".isalnum()


True

In [24]:
# server_url = "http://192.168.1.2:8060/service"
# from grobid_quantities.quantities import QuantitiesClient
# client = QuantitiesClient(apiBase=server_url)

# responses = {}
# for key in data.keys():
#     responses[key] = client.process_text(data[key].txt[0])

# import json 

# for key in responses.keys():
#     json.dump(responses[key], open(key+".grobid","w"),indent = 3)

In [76]:
from bs4 import BeautifulSoup

import requests

page = requests.get("http://phrontistery.info/unit.html")

page.text

soup = BeautifulSoup(page.content, "html.parser")

words = soup.table.table.text.split("\r\n")[4:]

In [77]:
gazetteer = {}
for i in range(len(words)-1):
    if i % 2 == 0:
        gazetteer[words[i]] = words[i + 1]

In [78]:
gazetteer

{'aam': 'archaic unit of liquid measure ranging from 37 to 41 gallons of wine',
 'abampere': 'unit equivalent to 10 amperes',
 'abvolt': 'unit of electrical potential equal to one hundred millionth of a volt',
 'acre-breadth': 'old unit of length of 22 yards',
 'ampere': 'unit for measuring amount of electrical current',
 'angstrom': 'unit of one ten-billionth of a meter',
 'arpent': 'unit of land area slightly smaller than an acre',
 'astronomical unit': 'unit of distance equal to the distance from the earth to the sun',
 'bar': 'unit of pressure of one million dynes per square centimeter',
 'barleycorn': 'old unit of length equal to one-third inch',
 'barn': 'unit of area measurement of particles equal to 100 square femtometres',
 'baud': 'unit of data transmission speed of one signal event per second',
 'bel': 'unit of noise intensity equal to ten decibels',
 'butt': 'unit of volume equal to two hogsheads or 126 gallons',
 'calorie': 'unit of heat or heat-producing value',
 'candela

In [110]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_unusual_units_of_measurement")

page.text

soup = BeautifulSoup(page.content, "html.parser")
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of unusual units of measurement - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b3696dcf-8082-458a-a16a-8393dfcaf64a","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_unusual_units_of_measurement","wgTitle":"List of unusual units of measurement","wgCurRevisionId":978631766,"wgRevisionId":978631766,"wgArticleId":23685094,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: extra text: authors list","Webarchive template wayback links","CS1 maint: uses authors 

In [111]:
words = soup.body.ul.text.split("\n")

In [112]:
while("" in words):
    words.remove("")

In [114]:
twords = [w.split(" ")[1:] for w in words]

In [124]:
def hel(x):
    temp= ""
    for y in x:
        if temp != "":
            temp = temp + " " + y
        else:
            temp = y
    return temp

In [129]:
fwords = [hel(x) for x in twords]

In [130]:
fwords

['Length',
 'Rack unit',
 'Hand',
 'Light-nanosecond',
 'Metric foot',
 'Horse',
 'Boat length',
 'Football field (length)',
 'Block',
 "The Earth's circumference",
 'Earth-to-Moon distance',
 'Siriometer',
 'Area',
 'Barn',
 'Brass',
 'Square',
 "Cow's grass",
 'Football field (area)',
 'Morgen',
 'Countries, regions, and cities',
 'Wales',
 'The United States',
 'Other countries',
 'Volume',
 'Metric ounce',
 'Shot',
 'Board foot or super foot',
 'Hoppus foot',
 'Cubic ton',
 'Cord and rick',
 'Twenty-foot equivalent unit',
 'Acre-foot',
 'Casual units',
 'Flow rate',
 "Miner's inch",
 'Mass',
 'Bag of cement and bag mix',
 'Grave',
 'Jupiter',
 'Sun',
 'Time',
 'Light-distance',
 'Shake',
 'Jiffy',
 'Microfortnight',
 'Sidereal day',
 'Decimal time systems',
 'Month and year',
 'Sol',
 'Dog year',
 'Galactic year',
 'KerMetric time',
 'Moment',
 'Thirds, fourths',
 'Angular measure',
 'Furman',
 'Binary degree, binary radian, brad',
 'Grade (also grad, gradian, gon)',
 'Mils, streck

In [132]:
for w in fwords: 
    if w not in ["Length",'Area','Volume','Flow rate', 'Mass', 'Time', 'Angular measure','Energy', 'Other metric-compatible scales', 'Demography and epidemiology', 'Computer and information science','Purchasing power parity', 'Other']:
        gazetteer[w] = None

In [133]:
gazetteer

{'aam': 'archaic unit of liquid measure ranging from 37 to 41 gallons of wine',
 'abampere': 'unit equivalent to 10 amperes',
 'abvolt': 'unit of electrical potential equal to one hundred millionth of a volt',
 'acre-breadth': 'old unit of length of 22 yards',
 'ampere': 'unit for measuring amount of electrical current',
 'angstrom': 'unit of one ten-billionth of a meter',
 'arpent': 'unit of land area slightly smaller than an acre',
 'astronomical unit': 'unit of distance equal to the distance from the earth to the sun',
 'bar': 'unit of pressure of one million dynes per square centimeter',
 'barleycorn': 'old unit of length equal to one-third inch',
 'barn': 'unit of area measurement of particles equal to 100 square femtometres',
 'baud': 'unit of data transmission speed of one signal event per second',
 'bel': 'unit of noise intensity equal to ten decibels',
 'butt': 'unit of volume equal to two hogsheads or 126 gallons',
 'calorie': 'unit of heat or heat-producing value',
 'candela

In [136]:
gazetteer.pop("See also")
gazetteer.pop("References")
gazetteer.pop("Bibliography")

In [137]:
gazetteer

{'aam': 'archaic unit of liquid measure ranging from 37 to 41 gallons of wine',
 'abampere': 'unit equivalent to 10 amperes',
 'abvolt': 'unit of electrical potential equal to one hundred millionth of a volt',
 'acre-breadth': 'old unit of length of 22 yards',
 'ampere': 'unit for measuring amount of electrical current',
 'angstrom': 'unit of one ten-billionth of a meter',
 'arpent': 'unit of land area slightly smaller than an acre',
 'astronomical unit': 'unit of distance equal to the distance from the earth to the sun',
 'bar': 'unit of pressure of one million dynes per square centimeter',
 'barleycorn': 'old unit of length equal to one-third inch',
 'barn': 'unit of area measurement of particles equal to 100 square femtometres',
 'baud': 'unit of data transmission speed of one signal event per second',
 'bel': 'unit of noise intensity equal to ten decibels',
 'butt': 'unit of volume equal to two hogsheads or 126 gallons',
 'calorie': 'unit of heat or heat-producing value',
 'candela

In [165]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get("https://www.adducation.info/how-to-improve-your-knowledge/units-of-measurement/", headers = headers)

page.text

soup = BeautifulSoup(page.content, "html.parser")

soup.body.div




<div class="navbar navbar-default navbar-top" role="navigation">
<div class="container-fluid">
<div class="row">
<div class="col-sm-12 col-md-4 col-lg-3">
<div class="navbar-header">
<div class="row">
<div class="col-xs-8">
<a class="navbar-brand" href="/"><img alt="Adducation Facts and Figures in Lists and Tables" src="/wp-content/themes/taschenhirn-2015/img/logo.png"/></a>
</div>
<div class="col-xs-4">
<button aria-controls="navbar" aria-expanded="false" class="navbar-toggle collapsed" data-target="#main-nav-collapse" data-toggle="collapse" type="button">
<span class="sr-only">Toggle navigation</span>
<span class="menu-label">▼ Menu</span>
</button>
</div>
</div>
</div>
</div>
<div class="col-sm-12 col-md-8 col-lg-9">
<div class="clearfix">
<div class="pull-right" style="width: 100%">
<div class="clearfix">
<div class="menu-service-container"><ul class="menu" id="menu-service"><li class="taschenhirn menu-item menu-item-type-custom menu-item-object-custom menu-item-22466" id="menu-ite

In [185]:
for x in soup.find_all("strong"):
    if x.text.isalpha():
        gazetteer[x.text] = None

In [187]:
len(gazetteer)

344

In [189]:
f = open("measurement.gazetteer", "w")
for key in gazetteer.keys():
    f.write(key+"\n")
f.close()

In [193]:
curatedGazz = []

f = open("measurement.gazetteer", "r")
for line in f: 
    curatedGazz.append(line.strip("\n"))
    

In [194]:
f.close()

In [195]:
curatedGazz

['aam',
 'abampere',
 'abvolt',
 'acre-breadth',
 'ampere',
 'angstrom',
 'arpent',
 'astronomical unit',
 'bar',
 'barleycorn',
 'barn',
 'baud',
 'bel',
 'butt',
 'calorie',
 'candela',
 'carat',
 'centner',
 'chain',
 'chaldron',
 'clove',
 'cord',
 'coulomb',
 'cubit',
 'curie',
 'cusec',
 'dalton',
 'darcy',
 'dash',
 'decibel',
 'denier',
 'digit',
 'dioptre',
 'dol',
 'dram',
 'dyne',
 'elephant',
 'ell',
 'em',
 'erg',
 'face-cord',
 'farad',
 'fathom',
 'fermi',
 'fifth',
 'firkin',
 'footlambert',
 'fresnel',
 'furlong',
 'gal',
 'gamma',
 'gauss',
 'gilbert',
 'gill',
 'grain',
 'gray',
 'hand',
 'henry',
 'hertz',
 'hide',
 'hobbet',
 'hogshead',
 'horsepower',
 'hundredweight',
 'jansky',
 'jiffy',
 'joule',
 'karat',
 'kelvin',
 'kilderkin',
 'kilobase',
 'kip',
 'lambert',
 'langley',
 'league',
 'link',
 'lumen',
 'lux',
 'magneton',
 'maxwell',
 'mecate',
 'megadeath',
 'mho',
 'mil',
 'minim',
 'mnemon',
 'mole',
 'morgan',
 'morgen',
 'mutchkin',
 'nail',
 'neper',
 

In [217]:
page = requests.get("https://www.hobbyprojects.com/dictionary_of_units.html")

page.text

soup = BeautifulSoup(page.content, "html.parser")
unitsTemp = [ x.text for x in soup.find(id = "maintext").find_all("a")]

In [218]:
unitsTemp

['Length',
 'area',
 'volume or capacity',
 'mass',
 'temperature',
 'density, area',
 'density, line',
 'density, volume',
 'energy',
 'force',
 'torque',
 'fuel consumption',
 'mass per unit length',
 'mass per unit area',
 'mass per unit volume \n\t\t\t',
 'power',
 'pressure',
 'speed',
 'spread rate (by mass)',
 'spread rate (by volume)',
 'stress',
 'System International [SI]',
 'Derived Units of the S I',
 'Conventions of Usage in the S I',
 'Metric System of Measurements',
 'Metrication in the UK',
 'The U S System of Measurements',
 'Definitions of the Seven Basic S I Units',
 'Prefixes of the S I',
 'A Brief History of Measurement',
 'The U K (Imperial) System of Measurements',
 '\nacres',
 'angstroms',
 'ares',
 'astronomical units',
 'atmospheres',
 '\nbarleycorns',
 'barrels (oil)',
 'bars',
 'british thermal units',
 'btu/hour etc.',
 'bushels',
 '\ncalories',
 'calories per hour etc.',
 'carats, metric',
 'celsius',
 'centigrade',
 'centigrade heat units',
 'centilitres'

In [219]:
import re

In [226]:
def clean(token):
    token = re.sub(r'[\(*\)]',"",token)
    token=token.replace("\n","")
    token=token.replace("\t","")
    token=token.replace("\r","")
    token=token.replace(".","")
    return token

In [227]:
unitsTemp = [clean(x) for x in unitsTemp]

In [228]:
unitsTemp

['Length',
 'area',
 'volume or capacity',
 'mass',
 'temperature',
 'density, area',
 'density, line',
 'density, volume',
 'energy',
 'force',
 'torque',
 'fuel consumption',
 'mass per unit length',
 'mass per unit area',
 'mass per unit volume ',
 'power',
 'pressure',
 'speed',
 'spread rate by mass',
 'spread rate by volume',
 'stress',
 'System International [SI]',
 'Derived Units of the S I',
 'Conventions of Usage in the S I',
 'Metric System of Measurements',
 'Metrication in the UK',
 'The U S System of Measurements',
 'Definitions of the Seven Basic S I Units',
 'Prefixes of the S I',
 'A Brief History of Measurement',
 'The U K Imperial System of Measurements',
 'acres',
 'angstroms',
 'ares',
 'astronomical units',
 'atmospheres',
 'barleycorns',
 'barrels oil',
 'bars',
 'british thermal units',
 'btu/hour etc',
 'bushels',
 'calories',
 'calories per hour etc',
 'carats, metric',
 'celsius',
 'centigrade',
 'centigrade heat units',
 'centilitres',
 'centimetres',
 'cent

In [229]:
curatedGazz = curatedGazz + unitsTemp

In [230]:
len(curatedGazz)

541

In [231]:
curatedGazz = list(dict.fromkeys(curatedGazz))

In [233]:
len(curatedGazz)

535

In [234]:
sorted(curatedGazz)

['A Brief History of Measurement',
 'ASTA pungency unit',
 'Acre',
 'Acre-foot',
 'Ampere',
 'Angular velocity',
 'Are',
 'Bag of cement and bag mix',
 'Banana equivalent dose',
 'Bar',
 'Barn',
 'Becquerel',
 'Big Mac Index',
 'Binary degree, binary radian, brad',
 'Block',
 'Board foot or super foot',
 'Boat length',
 'BogoMips',
 'Brass',
 'Calorie',
 'Candela',
 'Casual units',
 'Centigrade',
 'Centiliter',
 'Centimeter',
 'Centipawn',
 'Coffee/Latte',
 'Conventions of Usage in the S I',
 'Cord and rick',
 'Coulomb',
 'Countries, regions, and cities',
 "Cow's grass",
 'Cubic ton',
 'Cup',
 'DRY',
 'Data volume',
 'Day',
 'Decibel',
 'Deciliter',
 'Decimal time systems',
 'Decimeter',
 'Decitex',
 'Definitions of the Seven Basic S I Units',
 'Degree',
 'Denier',
 'Density',
 'Derived Units of the S I',
 'Diopter',
 'Dog year',
 'Dozen',
 'Dram',
 'Drivers',
 'Earth-to-Moon distance',
 'Electromagnetic flux',
 'Electronvolt mass',
 'Energy intensity',
 'FLOPS',
 'FLUID',
 'Fahrenheit

In [236]:
f = open("measurement.gazetteer", "w")
for key in curatedGazz:
    f.write(key+"\n")
f.close()

In [237]:
data

{'S0012821X12004384-1302': <__main__.exerpt at 0x20953bd1408>,
 'S0012821X12004384-1405': <__main__.exerpt at 0x20953bd1d48>,
 'S0012821X12004384-1415': <__main__.exerpt at 0x209536fc748>,
 'S0012821X12004384-1594': <__main__.exerpt at 0x20953bd1588>,
 'S0012821X12004384-1599': <__main__.exerpt at 0x20953bed908>,
 'S0012821X13002185-1061': <__main__.exerpt at 0x20953988c88>,
 'S0012821X13002185-1200': <__main__.exerpt at 0x20953bed348>,
 'S0012821X13002185-1217': <__main__.exerpt at 0x20953beef08>,
 'S0012821X13002185-1231': <__main__.exerpt at 0x20953beec88>,
 'S0012821X13002185-835': <__main__.exerpt at 0x20953bf5708>,
 'S0012821X13007309-1482': <__main__.exerpt at 0x20953bf3f48>,
 'S0012821X13007309-1509': <__main__.exerpt at 0x20953bf5508>,
 'S0012821X13007309-1605': <__main__.exerpt at 0x20953bfc888>,
 'S0012821X13007309-1989': <__main__.exerpt at 0x20953bfcbc8>,
 'S0016236113008041-3031': <__main__.exerpt at 0x20953bff488>,
 'S0016236113008041-3112': <__main__.exerpt at 0x20953c0

In [246]:
key = "S0016236113008041-3031"
data[key].txt

['With regards to elemental emissions from limestone use during the calcium looping cycle for CO2 capture, Dean [14] carried out experiments at the bench scale to investigate the effect of coal use on limestone-derived sorbent trace element inventory. Experiments carried out without coal use showed no change in sorbent trace element inventory. Batch experiments carried out using La Jagua coal showed an increase in the concentration of Ba, Cr, K, Mn, Sr, and Ti, whilst concentrations of B, Na and K remain the same, whilst for Cu there was a small decrease. For continuous experiments comparing 2 different coals and refuse-derived fuel (RDF), an increase in Al content was observed suggesting some ash mixed in with the sorbent. The concentrations of Ba, K, Sr and Zn remained the same for all fuels. For Lea Hall coal, there was an increase in B over eight cycles from ∼20 to 40 ppm, though not for La Jagua. For Cu, sorbent concentrations in the first cycle (∼15 ppm) remained the same over ei

In [247]:
data[key].tsv

,docId,annotSet,annotType,afterTag,beforeTag,afterContext,beforeContext,startOffset,endOffset,annotId,text,other
0,S0016236113008041-3031,1,Quantity,"[(different, JJ), (coals, NNS), (and, CC), (re...","[(a, DT), (small, JJ), (decrease, NN), (., .),...",different coals and refuse-derived fuel (RDF)...,a small decrease. For continuous experiments ...,597,598,T15,2,"{mods: [IsCount], unit: count}"
1,S0016236113008041-3031,1,MeasuredEntity,"[(and, CC), (refuse-derived, JJ), (fuel, NN), ...","[(small, JJ), (decrease, NN), (., .), (For, IN...","and refuse-derived fuel (RDF), an increase in",small decrease. For continuous experiments co...,599,614,T16,different coals,{HasQuantity: T15}
2,S0016236113008041-3031,2,Quantity,"[(cycles, NNS), (from, IN), (∼20, NN), (to, TO...","[(there, EX), (was, VBD), (an, DT), (increase,...","cycles from ∼20 to 40 ppm, though",there was an increase in B over,855,860,T17,eight,"{mods: [IsCount], unit: count}"
3,S0016236113008041-3031,2,MeasuredEntity,"[(from, IN), (∼20, NNP), (to, TO), (40, CD), (...","[(was, VBD), (an, DT), (increase, NN), (in, IN...","from ∼20 to 40 ppm, though not",was an increase in B over eight,861,867,T18,cycles,{HasQuantity: T17}
4,S0016236113008041-3031,3,Quantity,"[(,, ,), (though, IN), (not, RB), (for, IN), (...","[(an, DT), (increase, NN), (in, IN), (B, NNP),...",", though not for La Jagua. For Cu,",an increase in B over eight cycles,868,886,T1,from ∼20 to 40 ppm,"{mods: [IsApproximate, IsRange], unit: ppm}"
5,S0016236113008041-3031,3,MeasuredProperty,"[(over, IN), (eight, CD), (cycles, NNS), (from...","[(Hall, NNP), (coal, NN), (,, ,), (there, EX),...",over eight cycles from ∼20 to 40,"Hall coal, there was an increase in",848,849,T11,B,{HasQuantity: T1}
6,S0016236113008041-3031,3,MeasuredEntity,"[(use, NN), (during, IN), (the, DT), (calcium,...","[(With, IN), (regards, NNS), (to, TO), (elemen...",use during the calcium looping cycle for,With regards to elemental emissions from,41,50,T40,limestone,{HasProperty: T11}
7,S0016236113008041-3031,3,Qualifier,"[(from, IN), (∼20, NNP), (to, TO), (40, CD), (...","[(coal, NN), (,, ,), (there, EX), (was, VBD), ...","from ∼20 to 40 ppm, though not","coal, there was an increase in B",850,867,T9,over eight cycles,{Qualifies: T11}
8,S0016236113008041-3031,3,Qualifier,"[(in, IN), (B, NNP), (over, IN), (eight, CD), ...","[(For, IN), (Lea, NNP), (Hall, NNP), (coal, NN...",in B over eight cycles from ∼20,"For Lea Hall coal, there was an",836,844,T34,increase,{Qualifies: T1}
9,S0016236113008041-3031,3,Qualifier,"[(,, ,), (there, EX), (was, VBD), (an, DT), (i...","[(remained, VBD), (the, DT), (same, JJ), (for,...",", there was an increase in B over",remained the same for all fuels. For,808,821,T10,Lea Hall coal,{Qualifies: T1}


In [245]:
qText

[[('Five', 'CD')],
 [('2619.60', 'CD'),
  (',', ','),
  ('2617.35', 'CD'),
  (',', ','),
  ('2617.44', 'CD'),
  (',', ','),
  ('2614.73', 'CD'),
  (',', ','),
  ('and', 'CC'),
  ('2614.71', 'CD'),
  ('m', 'NN')],
 [('2619.60', 'CD'),
  (',', ','),
  ('2614.73', 'CD'),
  (',', ','),
  ('and', 'CC'),
  ('2614.71', 'CD'),
  ('m', 'NN')],
 [('two', 'CD')],
 [('2619.60', 'CD'), ('and', 'CC'), ('2614.71', 'CD')],
 [('20', 'CD')],
 [('0', 'CD'), ('%', 'NN'), ('to', 'TO'), ('80', 'CD'), ('%', 'NN')],
 [('from', 'IN'), ('2632', 'CD'), ('to', 'TO'), ('2618', 'CD'), ('m', 'NN')],
 [('on', 'IN'), ('average', 'JJ'), ('5', 'CD'), ('%', 'NN')],
 [('above', 'IN'), ('2618', 'CD'), ('m', 'NN')],
 [('40°N', 'CD')],
 [('from', 'IN'), ('103', 'CD'), ('yrs', 'NNS')],
 [('above', 'IN'), ('2618', 'CD'), ('m', 'NN')],
 [('∼33.7', 'NN'), ('Ma', 'NNP')],
 [('between', 'IN'),
  ('60', 'CD'),
  ('%', 'NN'),
  ('and', 'CC'),
  ('100', 'CD'),
  ('%', 'NN')],
 [('∼400', 'NN'), ('ka', 'NN')],
 [('million-year', 'JJ')]

In [248]:
key = "S0012821X13002185-1231"
data[key].txt

['Based on culture experiments (Sutton et al., 2013), it has recently been suggested that the diatom silicon isotope fractionation factor may be species dependent, and in particular that the genus Chaetoceros may have a significantly larger, and the species Fragilariopsis kerguelensis, a significantly smaller 30ε than the widely accepted value of ∼−1‰ (De la Rocha et al., 1997). Our previous Southern Ocean core top study included Antarctic Peninsula sediments containing Chaetoceros sp. and Pacific sector sediments containing F. kerguelensis (Egan et al., 2012). The size fractions analysed had up to 40% differences in the abundance of these diatoms (Fig. 5). However, there is no detectable δ30Si offset between size fractions in the core tops TAN127 or 385 (Fig. 5a and b), whilst in KC08A (Fig. 5c), there is a slight negative offset of the 12–20 μm fraction which is opposite to that expected if the abundance of Chaetoceros was the cause (the suggested larger fractionation would act to lo

In [249]:
data[key].tsv

,docId,annotSet,annotType,afterTag,beforeTag,afterContext,beforeContext,startOffset,endOffset,annotId,text,other
0,S0012821X13002185-1231,1,Quantity,"[((, (), (De, NNP), (la, NNP), (Rocha, NNP), (...","[(30ε, CD), (than, IN), (the, DT), (widely, RB...","(De la Rocha et al., 1997). Our",30ε than the widely accepted value of,347,351,T13,∼−1‰,"{mods: [IsApproximate], unit: ‰}"
1,S0012821X13002185-1231,1,MeasuredProperty,"[(than, IN), (the, DT), (widely, RB), (accepte...","[(the, DT), (species, NNS), (Fragilariopsis, N...",than the widely accepted value of ∼−1‰,"the species Fragilariopsis kerguelensis, a si...",309,312,T12,30ε,{HasQuantity: T13}
2,S0012821X13002185-1231,1,MeasuredEntity,"[(may, MD), (be, VB), (species, NNS), (depende...","[(it, PRP), (has, VBZ), (recently, RB), (been,...","may be species dependent, and in particular",it has recently been suggested that the,92,135,T11,diatom silicon isotope fractionation factor,{HasProperty: T12}
3,S0012821X13002185-1231,2,Quantity,"[(differences, NNS), (in, IN), (the, DT), (abu...","[(The, DT), (size, NN), (fractions, NNS), (ana...",differences in the abundance of these diatoms,The size fractions analysed had up to,604,607,T1,40%,{unit: ‰}
4,S0012821X13002185-1231,2,MeasuredProperty,"[(of, IN), (these, DT), (diatoms, NNS), ((, ()...","[(size, NN), (fractions, NNS), (analysed, VBD)...","of these diatoms (Fig. 5). However, there",size fractions analysed had up to 40%,608,636,T10,differences in the abundance,{HasQuantity: T1}
5,S0012821X13002185-1231,2,MeasuredEntity,"[((, (), (Fig, NNP), (., .), (5, CD), (), )), ...","[(40, CD), (%, NN), (differences, NNS), (in, I...","(Fig. 5). However, there is no detectable",40% differences in the abundance of these,646,653,T6,diatoms,{HasProperty: T10}
6,S0012821X13002185-1231,3,Quantity,"[(fraction, NN), (which, WDT), (is, VBZ), (opp...","[(is, VBZ), (a, DT), (slight, JJ), (negative, ...",fraction which is opposite to that expected,is a slight negative offset of the,848,856,T2,12–20 μm,"{mods: [IsRange], unit: μm}"
7,S0012821X13002185-1231,3,MeasuredEntity,"[(which, WDT), (is, VBZ), (opposite, JJ), (to,...","[(slight, JJ), (negative, JJ), (offset, NN), (...",which is opposite to that expected if,slight negative offset of the 12–20 μm,857,865,T9,fraction,{HasQuantity: T2}
8,S0012821X13002185-1231,4,Quantity,"[(display, VB), (a, DT), (good, JJ), (correlat...","[(Rather, RB), (,, ,), (the, DT), (δ30Si, NNP)...","display a good correlation (r2=0.92, Fig. 2a)","Rather, the δ30Si values of size fractions",1083,1102,T3,between 2 and 20 μm,"{mods: [IsRange], unit: μm}"
9,S0012821X13002185-1231,4,MeasuredEntity,"[(between, IN), (2, CD), (and, CC), (20, CD), ...","[(size, NN), (fractions, NNS), (), )), (., .),...",between 2 and 20 μm display a,"size fractions). Rather, the δ30Si values of",1068,1082,T7,size fractions,{HasQuantity: T3}


In [3]:
d

[200,
 {'runtime': 849,
  'measurements': [{'type': 'value',
    'quantity': {'rawValue': 'three',
     'parsedValue': {'name': 'three',
      'numeric': 3,
      'structure': {'type': 'ALPHABETIC', 'formatted': 'three'},
      'parsed': 'three'},
     'offsetStart': 492,
     'offsetEnd': 497},
    'quantified': {'rawName': 'axes (describing',
     'normalizedName': 'ax',
     'offsetStart': 498,
     'offsetEnd': 514}},
   {'type': 'value',
    'quantity': {'rawValue': 'Five',
     'parsedValue': {'name': 'Five',
      'numeric': 5,
      'structure': {'type': 'ALPHABETIC', 'formatted': 'Five'},
      'parsed': 'Five'},
     'offsetStart': 553,
     'offsetEnd': 557},
    'quantified': {'rawName': 'samples from',
     'normalizedName': 'sample',
     'offsetStart': 558,
     'offsetEnd': 570}},
   {'type': 'listc',
    'quantities': [{'rawValue': '2619.60',
      'parsedValue': {'name': '2619.60',
       'numeric': 60,
       'structure': {'type': 'NUMBER', 'formatted': '60'},
      

In [10]:
import json
d = json.load(open("data/trial/grobid/S0012821X12004384-1302.grobid"))

for x in d[1]["measurements"]:
    if x["type"] == "value":
        print(x["quantity"]["offsetStart"])
    elif x["type"] == "listc":
        for q in x["quantities"]:
            print(q["offsetStart"])

492
553
588
597
606
615
628
769
778
791
1481
1493
1505
1992
2115
